In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sys, getopt
import csv
import pandas as pd
from collections import Counter
import re

In [2]:
import os
path = os.getcwd()
print(path)
os.chdir(r"C:\Users\stanma02\Desktop\Final Project\WebScraper and Data")
path = os.getcwd()
print(path)

C:\Users\stanma02\Desktop\Final Project\WebScraper and Data
C:\Users\stanma02\Desktop\Final Project\WebScraper and Data


In [3]:
def webscraper(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')

    all_tables = soup.findAll("tbody")
    results=soup.find(id="team_stats")
    results2=soup.find(id="team_stats_extra")

    data= results.get_text()
    data2=results2.get_text()

    x=re.split("\n|\t|\xa0—\xa0|Cards|\xa0",data)
    test_list = list(filter(None, x))
    x2=re.split("\n|\xa0|\t|—|\xa0—\xa0",data2)
    test_list2 = list(filter(None, x2))

    Cards= results.find_all(class_='cards')
    Homecards= list(filter (lambda x: x not in ["yellow_card","red_card"],Cards[0]))
    Awaycards= list(filter (lambda x: x not in ["yellow_card","red_card"],Cards[1]))
    d= pd.DataFrame(data=Counter(Homecards).values()).T.rename(columns={0:"YellowCards",1:"RedCards"})
    e= pd.DataFrame(data=Counter(Awaycards).values()).T.rename(columns={0:"YellowCards",1:"RedCards"})

    if len(d.columns) < 2 :
        if len(d.columns)==0:
            d= {'YellowCards':[0], 'RedCards': [0]}
            d= pd.DataFrame(d)
        if len(d.columns)==1 and d.columns == "YellowCards":
            d["RedCards"] = 0
        if len(d.columns)==1 and d.columns == "RedCards":
            d["YellowCards"] = 0
        else:
            d

    if len(e.columns) < 2 :
        if len(e.columns)==0:
            e= {'YellowCards':[0], 'RedCards': [0]}
            e= pd.DataFrame(e)
        if len(e.columns)==1 and e.columns == "YellowCards":
            e["RedCards"] = 0
        if len(e.columns)==1 and e.columns == "RedCards":
            e["YellowCards"] = 0
        else:
            e

    split_string = test_list[7].split(" of ", 1)
    split_string1 = test_list[12].split(" of ", 1)
    split_string3= test_list2[2].split("Fouls")
    split_string4= test_list2[8].split("Tackles")
    split_string5= test_list2[9].split("Interceptions")
    split_string6 = test_list[10].split(" of ", 1)
    split_string7 = test_list[15].split(" of ", 1)

    data3= [test_list[5],split_string6[0],split_string6[1], split_string7[0],split_string7[1],
      split_string3[1],split_string4[1],split_string5[1]]
    data4= [test_list[4],split_string[0],split_string[1], split_string1[0],split_string1[1],
      split_string3[0],split_string4[0],split_string5[0]]
    HomeDF=pd.DataFrame(data4).T.rename(columns= {0:'HomePossession', 1:'HomePassesCompleted', 2:'HomePassesAttempts', 3:'HomeShotsonTarget',
                                             4:'HomeShots', 5:'HomeFouls', 6:'HomeTackles', 
                                             7:'HomeInterceptions'})
    d.rename(columns= {'YellowCards': 'HomeYellow','RedCards':'HomeRed'}, inplace=True)
    HomeFinal=pd.concat([HomeDF,d], axis=1)
    AwayDF=pd.DataFrame(data3).T.rename(columns= {0:'AwayPossession', 1:'AwayPassesCompleted', 2:'AwayPassesAttempts', 3:'AwayShotsonTarget',
                                              4:'AwayShots', 5:'AwayFouls', 6:'AwayTackles', 
                                              7:'AwayInterceptions'})
    e.rename(columns= {'YellowCards': 'AwayYellow','RedCards':'AwayRed'}, inplace=True)
    AwayFinal=pd.concat([AwayDF,e], axis=1)
    MatchStats=pd.concat([HomeFinal,AwayFinal],axis=1)
    return MatchStats

In [4]:
df = pd.read_csv("16-17.csv",encoding='cp1252')
df[['HomeGoal','AwayGoal']]= df.Score.str.split("–",expand=True)
TotalMatchData=[]

for index, row in df.iterrows():
    MatchData= webscraper(row["URL"])
    TotalMatchData = pd.DataFrame(TotalMatchData).append(MatchData).reset_index(drop=True)

LastDF=pd.concat([df.drop(columns=['Score']),TotalMatchData],axis=1)

In [5]:
season1617=LastDF
season1617.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Season               306 non-null    object 
 1   Date                 306 non-null    object 
 2   Home                 306 non-null    object 
 3   Away                 306 non-null    object 
 4   Attendance           306 non-null    int64  
 5   Venue                306 non-null    object 
 6   URL                  306 non-null    object 
 7   HomeGoal             306 non-null    object 
 8   AwayGoal             306 non-null    object 
 9   HomePossession       306 non-null    object 
 10  HomePassesCompleted  306 non-null    object 
 11  HomePassesAttempts   306 non-null    object 
 12  HomeShotsonTarget    306 non-null    object 
 13  HomeShots            306 non-null    object 
 14  HomeFouls            306 non-null    object 
 15  HomeTackles          306 non-null    obj

In [6]:
new1617=season1617.iloc[:, :-1]

In [211]:
df = pd.read_csv("17-18.csv",encoding='cp1252')
df[['HomeGoal','AwayGoal']]= df.Score.str.split("–",expand=True)
TotalMatchData=[]

for index, row in df.iterrows():
    MatchData= webscraper(row["URL"])
    TotalMatchData = pd.DataFrame(TotalMatchData).append(MatchData).reset_index(drop=True)

LastDF=pd.concat([df.drop(columns=['Score']),TotalMatchData],axis=1)

In [212]:
season1718=LastDF
season1718.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Season               306 non-null    object
 1   Date                 306 non-null    object
 2   Home                 306 non-null    object
 3   Away                 306 non-null    object
 4   Attendance           306 non-null    int64 
 5   Venue                306 non-null    object
 6   URL                  306 non-null    object
 7   HomeGoal             306 non-null    object
 8   AwayGoal             306 non-null    object
 9   HomePossession       306 non-null    object
 10  HomePassesCompleted  306 non-null    object
 11  HomePassesAttempts   306 non-null    object
 12  HomeShotsonTarget    306 non-null    object
 13  HomeShots            306 non-null    object
 14  HomeFouls            306 non-null    object
 15  HomeTackles          306 non-null    object
 16  HomeInte

In [213]:
df = pd.read_csv("18-19.csv",encoding='cp1252')
df[['HomeGoal','AwayGoal']]= df.Score.str.split("–",expand=True)
TotalMatchData=[]

for index, row in df.iterrows():
    MatchData= webscraper(row["URL"])
    TotalMatchData = pd.DataFrame(TotalMatchData).append(MatchData).reset_index(drop=True)

LastDF=pd.concat([df.drop(columns=['Score']),TotalMatchData],axis=1)

In [214]:
season1819=LastDF
season1819.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Season               306 non-null    object 
 1   Date                 306 non-null    object 
 2   Home                 306 non-null    object 
 3   Away                 306 non-null    object 
 4   Attendance           306 non-null    int64  
 5   Venue                306 non-null    object 
 6   URL                  306 non-null    object 
 7   HomeGoal             306 non-null    object 
 8   AwayGoal             306 non-null    object 
 9   HomePossession       306 non-null    object 
 10  HomePassesCompleted  306 non-null    object 
 11  HomePassesAttempts   306 non-null    object 
 12  HomeShotsonTarget    306 non-null    object 
 13  HomeShots            306 non-null    object 
 14  HomeFouls            306 non-null    object 
 15  HomeTackles          306 non-null    obj

In [215]:
new1819=season1819.iloc[:, :-1]

In [216]:
df = pd.read_csv("19-20.csv",encoding='cp1252')
df[['HomeGoal','AwayGoal']]= df.Score.str.split("–",expand=True)
TotalMatchData=[]

for index, row in df.iterrows():
    MatchData= webscraper(row["URL"])
    TotalMatchData = pd.DataFrame(TotalMatchData).append(MatchData).reset_index(drop=True)

LastDF=pd.concat([df.drop(columns=['Score']),TotalMatchData],axis=1)

In [217]:
season1920=LastDF
season1920.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Season               306 non-null    object 
 1   Date                 306 non-null    object 
 2   Home                 306 non-null    object 
 3   Away                 306 non-null    object 
 4   Attendance           306 non-null    int64  
 5   Venue                306 non-null    object 
 6   URL                  306 non-null    object 
 7   HomeGoal             306 non-null    object 
 8   AwayGoal             306 non-null    object 
 9   HomePossession       306 non-null    object 
 10  HomePassesCompleted  306 non-null    object 
 11  HomePassesAttempts   306 non-null    object 
 12  HomeShotsonTarget    306 non-null    object 
 13  HomeShots            306 non-null    object 
 14  HomeFouls            306 non-null    object 
 15  HomeTackles          306 non-null    obj

In [218]:
new1920=season1920.iloc[:, :-1]

In [219]:
df = pd.read_csv("2021new.csv",encoding='cp1252')
df[['HomeGoal','AwayGoal']]= df.Score.str.split("–",expand=True)
TotalMatchData=[]

for index, row in df.iterrows():
    MatchData= webscraper(row["URL"])
    TotalMatchData = pd.DataFrame(TotalMatchData).append(MatchData).reset_index(drop=True)

LastDF=pd.concat([df.drop(columns=['Score']),TotalMatchData],axis=1)

In [220]:
season2021=LastDF
season2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Season               216 non-null    object
 1   Date                 216 non-null    object
 2   Home                 216 non-null    object
 3   Away                 216 non-null    object
 4   Attendance           216 non-null    int64 
 5   Venue                216 non-null    object
 6   URL                  216 non-null    object
 7   HomeGoal             216 non-null    object
 8   AwayGoal             216 non-null    object
 9   HomePossession       216 non-null    object
 10  HomePassesCompleted  216 non-null    object
 11  HomePassesAttempts   216 non-null    object
 12  HomeShotsonTarget    216 non-null    object
 13  HomeShots            216 non-null    object
 14  HomeFouls            216 non-null    object
 15  HomeTackles          216 non-null    object
 16  HomeInte

In [221]:
print(new1617.shape)
print(season1718.shape)
print(new1819.shape)
print(new1920.shape)
print(season2021.shape)

(306, 29)
(306, 29)
(306, 29)
(306, 29)
(216, 29)


In [223]:
frames = [new1617, season1718, new1819, new1920,season2021]
FinalData=pd.concat(frames)

In [224]:
FinalData

,Season,Date,Home,Away,Attendance,Venue,URL,HomeGoal,AwayGoal,HomePossession,...,AwayPossession,AwayPassesCompleted,AwayPassesAttempts,AwayShotsonTarget,AwayShots,AwayFouls,AwayTackles,AwayInterceptions,AwayYellow,AwayRed
0,16-17,8/26/2016,Bayern Munich,Werder Bremen,75000,Allianz Arena,https://fbref.com/en/matches/cd4ef1df/Bayern-M...,6,0,71%,...,30%,241,315,2,4,8,14,19,1,0
1,16-17,8/27/2016,Dortmund,Mainz 05,81360,Signal Iduna Park,https://fbref.com/en/matches/0a2c80cb/Dortmund...,2,1,74%,...,26%,157,252,4,12,20,27,25,3,0
2,16-17,8/27/2016,Augsburg,Wolfsburg,26172,WWK Arena,https://fbref.com/en/matches/b4a595cd/Augsburg...,0,2,46%,...,54%,391,474,6,12,17,15,9,2,0
3,16-17,8/27/2016,Hamburger SV,Ingolstadt 04,50107,Volksparkstadion,https://fbref.com/en/matches/58ae4021/Hamburge...,1,1,47%,...,53%,299,399,5,13,15,17,14,2,0
4,16-17,8/27/2016,Eint Frankfurt,Schalke 04,47000,Commerzbank-Arena,https://fbref.com/en/matches/75808fc7/Eintrach...,1,0,44%,...,56%,352,438,3,11,15,19,10,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,20-21,3/6/2021,Eint Frankfurt,Stuttgart,0,Commerzbank-Arena,https://fbref.com/en/matches/109588f5/Eintrach...,1,1,58%,...,42%,283,387,1,8,19,29,8,3,0
212,20-21,3/6/2021,Bayern Munich,Dortmund,0,Allianz Arena,https://fbref.com/en/matches/33b71362/Der-Klas...,4,2,66%,...,34%,267,370,3,4,17,20,28,1,0
213,20-21,3/7/2021,Köln,Werder Bremen,0,RheinEnergieSTADION,https://fbref.com/en/matches/92c099c1/Koln-Wer...,1,1,68%,...,32%,230,323,5,12,10,14,17,0,0
214,20-21,3/7/2021,Arminia,Union Berlin,0,SchücoArena,https://fbref.com/en/matches/450ce215/Arminia-...,0,0,40%,...,60%,577,692,3,8,6,15,9,0,0


In [226]:
FinalData.to_csv(r'C:\Users\stanma02\Desktop\FinalData.csv')